In [ ]:
import polars as pl
import networkx as nx
from math import sqrt
import time
import random
import rtsvg
rt = rtsvg.RACETrack()

# Create a graph
from linknode_graph_patterns import LinkNodeGraphPatterns
_patterns_ = LinkNodeGraphPatterns()
g          = _patterns_.createPattern('mesh')

from polars_spring_layout import PolarsSpringLayout

# Create the dataframe version of the graph
_lu_ = {'fm':[], 'to':[]}
for edge in g.edges: _lu_['fm'].append(edge[0]), _lu_['to'].append(edge[1])
df = pl.DataFrame(_lu_)

# Run the layout
t0    = time.time()
_psl_ = PolarsSpringLayout(g) # 0.60s for the mesh pattern ... python ref time below is 1.139s
t1    = time.time()
_pos_ = nx.spring_layout(g)   # 0.008s for the mesh pattern
t2    = time.time()
print(f'polars time: {t1-t0} | nx.spring_layout time: {t2-t1}')

# Make an animation
_svg_ = rt.graphLayoutSVGAnimation(_psl_.pos_history, g, r=1.0)
rt.tile([_svg_, rt.link(df, [('fm','to')], pos=_psl_.results()), rt.link(df, [('fm','to')], pos=_pos_)])

In [ ]:
#
# "Graph Drawing by Force-directed Placement"
# Fruchterman & Reingold
# Sofftware -- Practice and Experience, Vol. 21 (1 1), 1129-1164 (November 1991)
#
# Figure 1 (Force-directed Placement)
#
# Almost the original ... but removed the boundary constraints
#
pos_history = {}
for _node_ in g.nodes: pos_history[_node_] = []

t    = 3.0
W, H = 256, 256
pos  = {}
for _node_ in g.nodes(): pos[_node_] = random.random()*W, random.random()*H
area = W*H
C   = 0.2 # paper says that C is found experimentally...
k   = C*sqrt(area/len(g.nodes()))
def f_a(z): return  z**2 / k # pseudocode says "x" vs "z" ... paper comments say "z"
def f_r(z): return  k**2 / z # paper comments differ on the sign

t0 = time.time()
_iterations_ = 2*len(g.nodes())
for i in range(_iterations_):
    for _node_ in g.nodes: pos_history[_node_].append(pos[_node_])
    disp = {}
    # calculate repulsive forces
    for v in g.nodes():
        disp[v] = 0.0, 0.0
        for u in g.nodes():
            if u == v or g.has_edge(u, v): continue
            _diff_     = pos[v][0] - pos[u][0], pos[v][1] - pos[u][1]
            _diff_len_ = max(sqrt(_diff_[0]**2 + _diff_[1]**2), 0.001)
            disp[v]    = disp[v][0] + (_diff_[0]/_diff_len_) * f_r(_diff_len_), disp[v][1] + (_diff_[1]/_diff_len_) * f_r(_diff_len_)
    # calculate the attractive forces
    for e in g.edges():
        e_v, e_u, w = e[0], e[1], 1 if 'weight' not in g[e[0]][e[1]] else g[e[0]][e[1]]['weight']
        _diff_     = pos[e_v][0] - pos[e_u][0], pos[e_v][1] - pos[e_u][1]
        _diff_len_ = max(sqrt(_diff_[0]**2 + _diff_[1]**2), 0.001)
        disp[e_v]  = disp[e_v][0] - (_diff_[0]/(_diff_len_*w)) * f_a(_diff_len_), disp[e_v][1] - (_diff_[1]/(_diff_len_*w)) * f_a(_diff_len_)
        disp[e_u]  = disp[e_u][0] + (_diff_[0]/(_diff_len_*w)) * f_a(_diff_len_), disp[e_u][1] + (_diff_[1]/(_diff_len_*w)) * f_a(_diff_len_)
    # limit the maximum displacement to the temperature t
    # ... and then prevent from being displaced outside frame
    for v in g.nodes():
        disp_len   = max(sqrt(disp[v][0]**2 + disp[v][1]**2), 0.001)
        xmag, ymag = t if abs(disp[v][0]) > t else disp[v][0], t if abs(disp[v][1]) > t else disp[v][1]
        pos[v] = pos[v][0] + (disp[v][0]/disp_len)*xmag, pos[v][1] + (disp[v][1]/disp_len)*ymag
    t *= 0.99 # t = cool(t)

for _node_ in g.nodes: pos_history[_node_].append(pos[_node_])

x0, y0, x1, y1 = pos[v][0], pos[v][1], pos[v][0], pos[v][1]
for k in pos_history.keys():
    x0, y0 = min(x0, min(pos_history[k][0])), min(y0, min(pos_history[k][1]))
    x1, y1 = max(x1, max(pos_history[k][0])), max(y1, max(pos_history[k][1]))

t1 = time.time()

print(f'python ref time: {t1-t0}')

# Convert it to an animation
_dur_ = '5s'
svg = [f'<svg x="0" y="0" width="512" height="512" viewBox="{x0} {y0} {x1-x0} {y1-y0}">']
svg.append(f'<rect x="{x0}" y="{y0}"  width="{x1-x0}" height="{y1-y0}" fill="white" />')
for _edge_ in g.edges:
    u, v = _edge_[0], _edge_[1]
    svg.append(f'<line x1="{pos[u][0]}" y1="{pos[u][1]}" x2="{pos[v][0]}" y2="{pos[v][1]}" stroke="black" stroke-width="0.2">')
    svg.append(f'<animate attributeName="x1" values="{";".join([f"{_pos_[0]}" for _pos_ in pos_history[u]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="y1" values="{";".join([f"{_pos_[1]}" for _pos_ in pos_history[u]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="x2" values="{";".join([f"{_pos_[0]}" for _pos_ in pos_history[v]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="y2" values="{";".join([f"{_pos_[1]}" for _pos_ in pos_history[v]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append('</line>')
for _node_ in g.nodes:
    svg.append(f'<circle cx="{pos_history[_node_][0][0]}" cy="{pos_history[_node_][0][1]}" r="1.2" fill="blue">')
    svg.append(f'<animate attributeName="cx" values="{";".join([f"{_pos_[0]}" for _pos_ in pos_history[_node_]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append(f'<animate attributeName="cy" values="{";".join([f"{_pos_[1]}" for _pos_ in pos_history[_node_]])}" dur="{_dur_}" repeatCount="indefinite" />')
    svg.append('</circle>')
svg.append('</svg>')

_lu_ = {'fm':[], 'to':[]}
for edge in g.edges: _lu_['fm'].append(edge[0]), _lu_['to'].append(edge[1])
df = pl.DataFrame(_lu_)
rt.tile([''.join(svg), rt.link(df, [('fm','to')],pos)])